In [161]:
import os
import pandas as pd
from datetime import datetime

# Remit

In [227]:
# Ordnerpfad zu den CSV-Dateien
csv_folder = 'artifacts/raw_data/Json_Dateien'

# Leere Liste zum Speichern der DataFrames
dataframes = []

# Alle CSV-Dateien im Ordner einlesen
for filename in os.listdir(csv_folder):
    if filename.endswith('.csv'):
        csv_file = os.path.join(csv_folder, filename)  # Vollständiger Pfad zur CSV-Datei
        df = pd.read_csv(csv_file)  # Einlesen der CSV-Datei
        dataframes.append(df)  # Hinzufügen des DataFrames zur Liste

# Alle DataFrames zu einem einzigen DataFrame zusammenführen
df_remit = pd.concat(dataframes, ignore_index=True)

In [196]:
df_remit

,id,dataset,mrid,revisionNumber,publishTime,createdTime,messageType,messageHeading,eventType,unavailabilityType,...,fuelType,normalCapacity,availableCapacity,unavailableCapacity,eventStatus,eventStartTime,eventEndTime,cause,relatedInformation,outageProfile
0,60924,REMIT,11XDONG-PT-----2-NGET-RMT-00001020,2,2020-11-26T13:54:00Z,2020-11-26T13:54:00Z,UnavailabilitiesOfElectricityFacilities,REMIT Information,Production unavailability,Planned,...,Wind Offshore,400,0,400,Active,2020-11-26T09:00:00Z,2020-11-26T18:00:00Z,Planned Outage,HOW01 Z12 Dry run interlink test,"[{'startTime': '2020-11-26T09:00:00Z', 'endTim..."
1,185054,REMIT,11XDONG-PT-----2-NGET-RMT-00001024,2,2020-12-10T16:12:00Z,2020-12-10T16:12:00Z,UnavailabilitiesOfElectricityFacilities,REMIT Information,Production unavailability,Planned,...,Wind Offshore,400,0,400,Active,2020-12-11T06:00:00Z,2020-12-11T16:00:00Z,Planned Outage,HOW01 BMU 2 will be undergoing frequency respo...,"[{'startTime': '2020-12-11T06:00:00Z', 'endTim..."
2,188013,REMIT,11XDONG-PT-----2-NGET-RMT-00001026,1,2020-12-13T19:13:00Z,2020-12-13T19:13:00Z,UnavailabilitiesOfElectricityFacilities,REMIT Information,Production unavailability,Planned,...,Wind Offshore,400,0,400,Active,2020-12-14T07:30:00Z,2020-12-14T09:30:00Z,Planned Outage,HOW01 BMU 2 will be undergoing frequency respo...,"[{'startTime': '2020-12-14T07:30:00Z', 'endTim..."
3,189243,REMIT,11XDONG-PT-----2-NGET-RMT-00001024,1,2020-12-07T21:19:00Z,2020-12-07T21:19:00Z,UnavailabilitiesOfElectricityFacilities,REMIT Information,Production unavailability,Planned,...,Wind Offshore,400,0,400,Active,2020-12-11T08:00:00Z,2020-12-11T18:00:00Z,Planned Outage,HOW01 BMU 2 will be undergoing frequency respo...,"[{'startTime': '2020-12-11T08:00:00Z', 'endTim..."
4,189474,REMIT,11XDONG-PT-----2-NGET-RMT-00001026,2,2020-12-14T09:29:00Z,2020-12-14T09:29:00Z,UnavailabilitiesOfElectricityFacilities,REMIT Information,Production unavailability,Planned,...,Wind Offshore,400,0,400,Active,2020-12-14T07:30:00Z,2020-12-14T10:30:00Z,Planned Outage,HOW01 BMU 2 will be undergoing frequency respo...,"[{'startTime': '2020-12-14T07:30:00Z', 'endTim..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431,783460,REMIT,11XDONG-PT-----2-NGET-RMT-00002044,2,2024-09-19T08:33:30Z,2024-09-19T08:29:00Z,UnavailabilitiesOfElectricityFacilities,REMIT Information,Production unavailability,Unplanned,...,Wind Offshore,400,0,400,Active,2024-09-18T17:14:00Z,2024-09-20T17:00:00Z,Under Investigation,Production unavailability due to restriction o...,"[{'startTime': '2024-09-18T17:14:00Z', 'endTim..."
432,783620,REMIT,11XDONG-PT-----2-NGET-RMT-00002044,3,2024-09-20T11:51:24Z,2024-09-20T11:44:00Z,UnavailabilitiesOfElectricityFacilities,REMIT Information,Production unavailability,Unplanned,...,Wind Offshore,400,0,400,Active,2024-09-18T17:14:00Z,2024-09-20T15:00:00Z,Under Investigation,Production unavailability due to restriction o...,"[{'startTime': '2024-09-18T17:14:00Z', 'endTim..."
433,788391,REMIT,11XDONG-PT-----2-NGET-RMT-00002040,2,2024-09-26T09:57:50Z,2024-09-26T09:53:00Z,UnavailabilitiesOfElectricityFacilities,REMIT Information,Production unavailability,Planned,...,Wind Offshore,400,200,200,Active,2024-09-23T08:00:00Z,2024-09-29T12:00:00Z,Planned Outage,NaN,"[{'startTime': '2024-09-23T08:00:00Z', 'endTim..."
434,788800,REMIT,11XDONG-PT-----2-NGET-RMT-00002055,1,2024-09-29T16:45:46Z,2024-09-29T16:42:00Z,UnavailabilitiesOfElectricityFacilities,REMIT Information,Production unavailability,Unplanned,...,Wind Offshore,400,250,150,Active,2024-09-29T16:30:00Z,2024-09-30T16:30:00Z,Under Investigation,NaN,"[{'startTime': '2024-09-29T16:30:00Z', 'endTim..."


In [228]:
def round_to_nearest_half_hour(dt):
    if dt.minute < 15:
        return dt.replace(minute=0, second=0)
    elif dt.minute < 45:
        return dt.replace(minute=30, second=0)
    else:
        return dt.replace(minute=0, second=0) + pd.Timedelta(hours=1)

In [229]:
df_remit = df_remit[["unavailabilityType", "normalCapacity", "availableCapacity", "eventStartTime", "eventEndTime"]]
df_remit = df_remit.assign(
    eventStartTime=pd.to_datetime(df_remit["eventStartTime"]),
    eventEndTime=pd.to_datetime(df_remit["eventEndTime"])
)
df_remit["eventStartTime"] = df_remit["eventStartTime"].apply(round_to_nearest_half_hour)
df_remit["eventEndTime"] = df_remit["eventEndTime"].apply(round_to_nearest_half_hour) 

In [230]:
df_remit = df_remit[df_remit["unavailabilityType"] == "Planned"]
df_remit#.groupby(by="eventStartTime").

,unavailabilityType,normalCapacity,availableCapacity,eventStartTime,eventEndTime
0,Planned,400,0,2020-11-26 09:00:00+00:00,2020-11-26 18:00:00+00:00
1,Planned,400,0,2020-12-11 06:00:00+00:00,2020-12-11 16:00:00+00:00
2,Planned,400,0,2020-12-14 07:30:00+00:00,2020-12-14 09:30:00+00:00
3,Planned,400,0,2020-12-11 08:00:00+00:00,2020-12-11 18:00:00+00:00
4,Planned,400,0,2020-12-14 07:30:00+00:00,2020-12-14 10:30:00+00:00
...,...,...,...,...,...
424,Planned,400,250,2024-06-12 11:00:00+00:00,2024-06-12 16:00:00+00:00
427,Planned,400,0,2024-06-20 08:00:00+00:00,2024-06-20 16:00:00+00:00
428,Planned,400,0,2024-08-30 07:00:00+00:00,2024-08-30 09:00:00+00:00
429,Planned,400,200,2024-09-23 08:00:00+00:00,2024-09-30 16:00:00+00:00


In [42]:
new_range = pd.date_range(df_remit["eventStartTime"][0], df_remit["eventEndTime"][0], freq="30min")
new_range

DatetimeIndex(['2020-11-26 09:00:00+00:00', '2020-11-26 09:30:00+00:00',
               '2020-11-26 10:00:00+00:00', '2020-11-26 10:30:00+00:00',
               '2020-11-26 11:00:00+00:00', '2020-11-26 11:30:00+00:00',
               '2020-11-26 12:00:00+00:00', '2020-11-26 12:30:00+00:00',
               '2020-11-26 13:00:00+00:00', '2020-11-26 13:30:00+00:00',
               '2020-11-26 14:00:00+00:00', '2020-11-26 14:30:00+00:00',
               '2020-11-26 15:00:00+00:00', '2020-11-26 15:30:00+00:00',
               '2020-11-26 16:00:00+00:00', '2020-11-26 16:30:00+00:00',
               '2020-11-26 17:00:00+00:00', '2020-11-26 17:30:00+00:00',
               '2020-11-26 18:00:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='30min')

In [58]:
t1 = df_remit[0:1].set_index("eventStartTime").reindex(new_range, method="nearest")

In [70]:
new_range = pd.date_range(df_remit["eventStartTime"][1], df_remit["eventEndTime"][1], freq="30min")
t2 = df_remit.loc[[1]].set_index("eventStartTime").reindex(new_range, method="nearest")

In [84]:
# pd.concat([t1, t2], axis=0)
df_remit

,publishTime,eventType,unavailabilityType,normalCapacity,availableCapacity,eventStartTime,eventEndTime,cause
0,2020-11-26 13:54:00+00:00,Production unavailability,Planned,400,0,2020-11-26 09:00:00+00:00,2020-11-26 18:00:00+00:00,Planned Outage
1,2020-12-10 16:12:00+00:00,Production unavailability,Planned,400,0,2020-12-11 06:00:00+00:00,2020-12-11 16:00:00+00:00,Planned Outage
2,2020-12-13 19:13:00+00:00,Production unavailability,Planned,400,0,2020-12-14 07:30:00+00:00,2020-12-14 09:30:00+00:00,Planned Outage
3,2020-12-07 21:19:00+00:00,Production unavailability,Planned,400,0,2020-12-11 08:00:00+00:00,2020-12-11 18:00:00+00:00,Planned Outage
4,2020-12-14 09:29:00+00:00,Production unavailability,Planned,400,0,2020-12-14 07:30:00+00:00,2020-12-14 10:30:00+00:00,Planned Outage
...,...,...,...,...,...,...,...,...
431,2024-09-19 08:33:30+00:00,Production unavailability,Unplanned,400,0,2024-09-18 17:14:00+00:00,2024-09-20 17:00:00+00:00,Under Investigation
432,2024-09-20 11:51:24+00:00,Production unavailability,Unplanned,400,0,2024-09-18 17:14:00+00:00,2024-09-20 15:00:00+00:00,Under Investigation
433,2024-09-26 09:57:50+00:00,Production unavailability,Planned,400,200,2024-09-23 08:00:00+00:00,2024-09-29 12:00:00+00:00,Planned Outage
434,2024-09-29 16:45:46+00:00,Production unavailability,Unplanned,400,250,2024-09-29 16:30:00+00:00,2024-09-30 16:30:00+00:00,Under Investigation


In [243]:
expanded_rows = []
for idx, row in df_remit.iterrows():
    new_range = pd.date_range(
        df_remit["eventStartTime"][idx], 
        df_remit["eventEndTime"][idx], 
        freq="30min"
    )
    expended_df = (
        row.to_frame().T
        .set_index("eventStartTime")
        .reindex(new_range, method="nearest")
        .reset_index()
        .rename(columns={"index": "eventStartTime"})
    )
    expanded_rows.append(expended_df)

result_df = pd.concat(expanded_rows, axis=0)

c:\Tom\HKA\7_Semester\Domänenprojekt_2\DoPro\.env\Lib\site-packages\pandas\core\indexes\base.py:7588: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(sequences[0], name=names)
c:\Tom\HKA\7_Semester\Domänenprojekt_2\DoPro\.env\Lib\site-packages\pandas\core\indexes\base.py:7588: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(sequences[0], name=names)
c:\Tom\HKA\7_Semester\Domänenprojekt_2\DoPro\.env\Lib\site-packages\pandas\core\indexes\base.py:7588: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Cal

In [247]:
# dates = pd.date_range(datetime(2020, 9, 20, 0, 0, 0), datetime(2024, 5, 19, 23, 30, 0))
result_df = result_df.loc[(result_df["eventStartTime"] <= "2024-05-20") & (result_df["eventStartTime"] >= "2020-09-20")]
result_df = result_df[["eventStartTime", "normalCapacity", "availableCapacity"]].groupby(by="eventStartTime").sum()

In [248]:
result_df

,normalCapacity,availableCapacity
eventStartTime,,
2020-09-23 09:00:00+00:00,1200,0
2020-09-23 09:30:00+00:00,1200,0
2020-09-23 10:00:00+00:00,1200,0
2020-09-23 10:30:00+00:00,1200,0
2020-09-23 11:00:00+00:00,1200,0
...,...,...
2024-05-19 22:00:00+00:00,7200,0
2024-05-19 22:30:00+00:00,7200,0
2024-05-19 23:00:00+00:00,7200,0


In [249]:
merged_df = pd.read_parquet(r"artifacts\prepared_data\merged_data.parquet")
merged_df

,reference_time,valid_time,RelativeHumidity,temp_hornsea,WindDirection,WindDirection:100,WindSpeed,WindSpeed:100,hours_after,CloudCover,...,season_summer,season_winter,wind_dir_cat_E,wind_dir_cat_N,wind_dir_cat_NE,wind_dir_cat_NW,wind_dir_cat_S,wind_dir_cat_SE,wind_dir_cat_SW,wind_dir_cat_W
0,2020-09-20 00:00:00+00:00,2020-09-20 00:00:00+00:00,85.213753,15.416670,61.588081,62.085171,10.043627,11.802604,0.0,0.450405,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,2020-09-20 00:00:00+00:00,2020-09-20 00:30:00+00:00,85.012268,15.412510,61.203667,61.726974,9.905537,11.648818,0.5,0.472211,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2020-09-20 00:00:00+00:00,2020-09-20 01:00:00+00:00,84.810783,15.408349,60.819256,61.368782,9.767446,11.495032,1.0,0.494018,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,2020-09-20 00:00:00+00:00,2020-09-20 01:30:00+00:00,84.357895,15.451218,60.511024,61.111046,9.631038,11.354128,1.5,0.520214,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,2020-09-20 00:00:00+00:00,2020-09-20 02:00:00+00:00,83.904999,15.494086,60.202793,60.853313,9.494630,11.213223,2.0,0.546410,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1273718,2024-05-19 00:00:00+00:00,2024-05-19 21:30:00+00:00,91.634125,11.190109,21.214108,21.933590,6.882957,8.258532,21.5,0.024549,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1273719,2024-05-19 00:00:00+00:00,2024-05-19 22:00:00+00:00,91.232635,11.127069,22.417616,22.716900,7.040593,8.282218,22.0,0.027738,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1273720,2024-05-19 00:00:00+00:00,2024-05-19 22:30:00+00:00,90.249329,11.237669,22.327656,22.829685,6.842084,8.272179,22.5,0.045711,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1273721,2024-05-19 00:00:00+00:00,2024-05-19 23:00:00+00:00,89.266022,11.348270,22.237694,22.942471,6.643575,8.262140,23.0,0.063685,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [250]:
new_df = merged_df.merge(result_df, how="left", left_on="valid_time", right_index=True)

In [251]:
new_df

,reference_time,valid_time,RelativeHumidity,temp_hornsea,WindDirection,WindDirection:100,WindSpeed,WindSpeed:100,hours_after,CloudCover,...,wind_dir_cat_E,wind_dir_cat_N,wind_dir_cat_NE,wind_dir_cat_NW,wind_dir_cat_S,wind_dir_cat_SE,wind_dir_cat_SW,wind_dir_cat_W,normalCapacity,availableCapacity
0,2020-09-20 00:00:00+00:00,2020-09-20 00:00:00+00:00,85.213753,15.416670,61.588081,62.085171,10.043627,11.802604,0.0,0.450405,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1,2020-09-20 00:00:00+00:00,2020-09-20 00:30:00+00:00,85.012268,15.412510,61.203667,61.726974,9.905537,11.648818,0.5,0.472211,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
2,2020-09-20 00:00:00+00:00,2020-09-20 01:00:00+00:00,84.810783,15.408349,60.819256,61.368782,9.767446,11.495032,1.0,0.494018,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
3,2020-09-20 00:00:00+00:00,2020-09-20 01:30:00+00:00,84.357895,15.451218,60.511024,61.111046,9.631038,11.354128,1.5,0.520214,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
4,2020-09-20 00:00:00+00:00,2020-09-20 02:00:00+00:00,83.904999,15.494086,60.202793,60.853313,9.494630,11.213223,2.0,0.546410,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1273718,2024-05-19 00:00:00+00:00,2024-05-19 21:30:00+00:00,91.634125,11.190109,21.214108,21.933590,6.882957,8.258532,21.5,0.024549,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,7200,0
1273719,2024-05-19 00:00:00+00:00,2024-05-19 22:00:00+00:00,91.232635,11.127069,22.417616,22.716900,7.040593,8.282218,22.0,0.027738,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,7200,0
1273720,2024-05-19 00:00:00+00:00,2024-05-19 22:30:00+00:00,90.249329,11.237669,22.327656,22.829685,6.842084,8.272179,22.5,0.045711,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,7200,0
1273721,2024-05-19 00:00:00+00:00,2024-05-19 23:00:00+00:00,89.266022,11.348270,22.237694,22.942471,6.643575,8.262140,23.0,0.063685,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,7200,0


In [13]:
df_remit['publishTime'] = pd.to_datetime(df_remit['publishTime'])
df_remit['createdTime'] = pd.to_datetime(df_remit['createdTime'])

# Versuche, 'eventStatus' und 'eventStartTime' in Datetime-Objekte zu konvertieren; ungültige Werte werden in NaT umgewandelt
df_remit['eventStatus'] = pd.to_datetime(df_remit['eventStatus'], errors='coerce')
df_remit['eventStartTime'] = pd.to_datetime(df_remit['eventStartTime'], errors='coerce')
df_remit['eventEndTime'] = pd.to_datetime(df_remit['eventEndTime'], errors='coerce')


df_remit['eventTime'] = df_remit['eventEndTime'] - df_remit['eventStartTime']

C:\Users\Michael Jäckle\AppData\Local\Temp\ipykernel_18260\998120984.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_remit['eventStatus'] = pd.to_datetime(df_remit['eventStatus'], errors='coerce')


In [14]:
df_remit = df_remit.filter(['publishTime','assetId','cause','eventTime'])
df_remit

,publishTime,assetId,cause,eventTime
0,2020-11-26 13:54:00+00:00,T_HOWAO-2,Planned Outage,0 days 09:00:00
1,2020-12-10 16:12:00+00:00,T_HOWAO-2,Planned Outage,0 days 10:00:00
2,2020-12-13 19:13:00+00:00,T_HOWAO-2,Planned Outage,0 days 02:00:00
3,2020-12-07 21:19:00+00:00,T_HOWAO-2,Planned Outage,0 days 10:00:00
4,2020-12-14 09:29:00+00:00,T_HOWAO-2,Planned Outage,0 days 03:00:00
...,...,...,...,...
431,2024-09-19 08:33:30+00:00,T_HOWAO-1,Under Investigation,1 days 23:46:00
432,2024-09-20 11:51:24+00:00,T_HOWAO-1,Under Investigation,1 days 21:46:00
433,2024-09-26 09:57:50+00:00,T_HOWAO-1,Planned Outage,6 days 04:00:00
434,2024-09-29 16:45:46+00:00,T_HOWAO-1,Under Investigation,1 days 00:00:00


In [112]:
# Funktion zum Runden auf das nächste 30-Minuten-Intervall
def round_to_nearest_half_hour(dt):
    if dt.minute < 15:
        return dt.replace(minute=0, second=0)
    elif dt.minute < 45:
        return dt.replace(minute=30, second=0)
    else:
        return dt.replace(minute=0, second=0) + pd.Timedelta(hours=1)

# Funktion zum Runden auf die nächste volle Stunde
def round_to_nearest_hour(dt):
    if dt.minute < 30:
        return dt.replace(minute=0, second=0)
    else:
        return dt.replace(minute=0, second=0) + pd.Timedelta(hours=1)

# Anwendung der Rundungsfunktionen auf df_tt
# df_remit['rounded_half_hour'] = df_remit['publishTime'].apply(round_to_nearest_half_hour)

# df_remit = df_remit.filter(['assetId',	'cause'	,'eventTime',	'rounded_half_hour'	])

In [16]:
df_remit

,assetId,cause,eventTime,rounded_half_hour
0,T_HOWAO-2,Planned Outage,0 days 09:00:00,2020-11-26 14:00:00+00:00
1,T_HOWAO-2,Planned Outage,0 days 10:00:00,2020-12-10 16:00:00+00:00
2,T_HOWAO-2,Planned Outage,0 days 02:00:00,2020-12-13 19:00:00+00:00
3,T_HOWAO-2,Planned Outage,0 days 10:00:00,2020-12-07 21:30:00+00:00
4,T_HOWAO-2,Planned Outage,0 days 03:00:00,2020-12-14 09:30:00+00:00
...,...,...,...,...
431,T_HOWAO-1,Under Investigation,1 days 23:46:00,2024-09-19 08:30:00+00:00
432,T_HOWAO-1,Under Investigation,1 days 21:46:00,2024-09-20 12:00:00+00:00
433,T_HOWAO-1,Planned Outage,6 days 04:00:00,2024-09-26 10:00:00+00:00
434,T_HOWAO-1,Under Investigation,1 days 00:00:00,2024-09-29 17:00:00+00:00
